In [1]:
# for running in collab, sagemaker etc.
import sys
import os


def _append_sys_path_of_utils():
    """
    Appends an utils directory to a system path so it is visible for the script.
    """
    UTILS_RELATIVE_PATH = "../../utils/"

    absolute_path_of_utils = os.path.abspath(UTILS_RELATIVE_PATH)
    if absolute_path_of_utils:
        if absolute_path_of_utils in sys.path:
            print(f"Absolute path already in sys.path: {absolute_path_of_utils}")
        else:
            print(f"Appended absolute path: {absolute_path_of_utils}")
            sys.path.insert(0, absolute_path_of_utils)
    else:
        print("Absolute path of utils module not found.")


_append_sys_path_of_utils()

Appended absolute path: /home/studio-lab-user/Generative_Models_for_CERN_Fast_Simulations/utils


In [1]:
import matplotlib

In [2]:
matplotlib.__version__

'3.6.2'

In [2]:
import wandb

In [3]:
!wandb login

wandb: Currently logged in as: bedkowski-patrick (nlp-wut-2023). Use `wandb login --relogin` to force relogin


In [4]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

2023-08-10 21:18:30.252967: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-08-10 21:18:30.253844: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-08-10 21:18:30.253852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, decomposition, manifold, preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, auc, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from tensorflow.keras import layers
import pickle
import time
from numpy import load
from matplotlib import pyplot
import pickle
import argparse

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

from tensorflow.compat.v1.keras.layers import Input, Dense, LeakyReLU, Conv2D, MaxPooling2D, UpSampling2D,  Concatenate
from tensorflow.compat.v1.keras.models import Model
from tensorflow.compat.v1.keras.layers import Dense, Reshape, Flatten
from tensorflow.compat.v1.keras.layers import Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import mse, binary_crossentropy, logcosh
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"

from scipy.stats import wasserstein_distance
import pandas as pd
from utils import sum_channels_parallel as sum_channels_parallel
from sklearn.metrics import mean_absolute_error
from datetime import datetime

2.11.0


In [6]:
SAVE_EXPERIMENT_DATA = True

### Load and process data

In [7]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display
import sklearn
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [8]:
data_proton = pd.read_pickle('../../data/data_proton_photonsum_n_15_2133_p_15_3273.pkl')
print('Loaded: ',  data_proton.shape, "max:", data_proton.max())

Loaded:  (48714, 56, 30) max: 678.0


In [9]:
data_neutron = pd.read_pickle('../../data/data_neutron_photonsum_n_15_2133_p_15_3273.pkl')
print('Loaded: ',  data_neutron.shape, "max:", data_neutron.max())

Loaded:  (48714, 44, 44) max: 532.0


In [10]:
data_cond = pd.read_pickle('../../data/data_cond_stddev_photonsum_p_15_2133_n_15_3273.pkl')
print('Loaded cond: ',  data_cond.shape, "max:", data_cond.values.max(), "min:", data_cond.values.min())

Loaded cond:  (48714, 15) max: 7000.0 min: -7000.0


In [11]:
# calculate min max proton, neutron sum
photon_sum_proton_min, photon_sum_proton_max = data_cond.proton_photon_sum.min(), data_cond.proton_photon_sum.max()
photon_sum_neutron_min, photon_sum_neutron_max = data_cond.neutron_photon_sum.min(), data_cond.neutron_photon_sum.max()

In [12]:
data_cond.drop(columns=['proton_photon_sum', 'neutron_photon_sum', 'group_number_proton', 'group_number_neutron'], inplace=True)
data_cond.columns, len(data_cond.columns)

(Index(['Energy', 'Vx', 'Vy', 'Vz', 'Px', 'Py', 'Pz', 'mass', 'charge',
        'std_proton', 'std_neutron'],
       dtype='object'),
 11)

In [13]:
data_cond.head()

,Energy,Vx,Vy,Vz,Px,Py,Pz,mass,charge,std_proton,std_neutron
0,3192.38,0.000000e+00,0.000000e+00,0.000000e+00,0.022422,-0.182957,-3192.38,939.565413,0.0,0.076311,0.632134
1,3961.55,0.000000e+00,0.000000e+00,0.000000e+00,-0.076487,0.179845,3961.55,938.272081,1.0,0.479201,0.127533
2,2770.10,1.861170e-17,2.517190e-17,-1.689330e-13,0.305187,0.412760,-2770.10,497.611000,0.0,0.090233,0.779230
3,3195.12,0.000000e+00,0.000000e+00,0.000000e+00,0.560528,-0.149980,3195.11,938.272081,1.0,0.131331,0.259529
4,1714.07,0.000000e+00,0.000000e+00,0.000000e+00,0.457768,0.145639,-1714.07,939.565413,0.0,0.103628,0.355864


In [14]:
STRENGTH = 0.1

DATE_STR = datetime.now().strftime("%d_%m_%Y_%H_%M")

NAME = "sdi-gan-padded"

wandb_run_name = f"{int(photon_sum_proton_min)}_{int(photon_sum_proton_max)}_{DATE_STR}"

EXPERIMENT_DIR_NAME = f"experiments/{NAME}_{int(photon_sum_proton_min)}_{int(photon_sum_proton_max)}_{DATE_STR}"

print("Experiment DIR: ", EXPERIMENT_DIR_NAME)

Experiment DIR:  experiments/sdi-gan-padded_15_2133_10_08_2023_21_18


In [15]:
def create_dir(path):
    if SAVE_EXPERIMENT_DATA:
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)

In [16]:
def save_scales(model_name, scaler_means, scaler_scales):
    out_fnm = f"{model_name}_scales.txt"
    res = "#means"
    for mean_ in scaler_means:
        res += "\n" + str(mean_)
    res += "\n\n#scales"
    for scale_ in scaler_scales:
        res += "\n" + str(scale_)

    if SAVE_EXPERIMENT_DATA:
        filepath = f"../../{EXPERIMENT_DIR_NAME}/scales/"
        create_dir(filepath)
        with open(filepath+out_fnm, mode="w") as f:
            f.write(res)

In [17]:
data_cond["cond"] = data_cond["Energy"].astype(str) +"|"+ data_cond["Vx"].astype(str) +"|"+  data_cond["Vy"].astype(str) +"|"+ data_cond["Vz"].astype(str) +"|"+  data_cond["Px"].astype(str) +"|"+  data_cond["Py"].astype(str) +"|"+ data_cond["Pz"].astype(str) +"|"+  data_cond["mass"].astype(str) +"|"+  data_cond["charge"].astype(str)
data_cond.head()

,Energy,Vx,Vy,Vz,Px,Py,Pz,mass,charge,std_proton,std_neutron,cond
0,3192.38,0.000000e+00,0.000000e+00,0.000000e+00,0.022422,-0.182957,-3192.38,939.565413,0.0,0.076311,0.632134,3192.38|0.0|0.0|0.0|0.022422400000000002|-0.18...
1,3961.55,0.000000e+00,0.000000e+00,0.000000e+00,-0.076487,0.179845,3961.55,938.272081,1.0,0.479201,0.127533,3961.55|0.0|0.0|0.0|-0.0764874|0.179845|3961.5...
2,2770.10,1.861170e-17,2.517190e-17,-1.689330e-13,0.305187,0.412760,-2770.10,497.611000,0.0,0.090233,0.779230,2770.1|1.86117e-17|2.5171900000000004e-17|-1.6...
3,3195.12,0.000000e+00,0.000000e+00,0.000000e+00,0.560528,-0.149980,3195.11,938.272081,1.0,0.131331,0.259529,3195.12|0.0|0.0|0.0|0.560528|-0.14998|3195.11|...
4,1714.07,0.000000e+00,0.000000e+00,0.000000e+00,0.457768,0.145639,-1714.07,939.565413,0.0,0.103628,0.355864,1714.07|0.0|0.0|0.0|0.457768|0.145639000000000...


In [18]:
data_cond_id = data_cond[["cond"]].reset_index()
data_cond_id

,index,cond
0,0,3192.38|0.0|0.0|0.0|0.022422400000000002|-0.18...
1,1,3961.55|0.0|0.0|0.0|-0.0764874|0.179845|3961.5...
2,2,2770.1|1.86117e-17|2.5171900000000004e-17|-1.6...
3,3,3195.12|0.0|0.0|0.0|0.560528|-0.14998|3195.11|...
4,4,1714.07|0.0|0.0|0.0|0.457768|0.145639000000000...
...,...,...
48709,48709,1456.66|7.79676e-18|2.26923e-18|-4.06504e-14|0...
48710,48710,3812.18|0.0|0.0|0.0|0.0529534|0.402807|-3812.1...
48711,48711,1422.39|1.1825599999999999e-05|-1.3857e-06|0.0...
48712,48712,4071.23|0.0|0.0|0.0|0.089305|0.0694104|-4071.2...


In [19]:
# select a random index of the same conditional data
# shuffle the data and merge it according to the conditional data. Pick the first index of the grouped conditional data
# if some unique conditional data has only one index of sample, then pair it with the same index 
ids = data_cond_id.merge(data_cond_id.sample(frac=1), on=["cond"], how="inner").groupby("index_x").first()
ids = ids["index_y"]
ids

index_x
0        12888
1        42140
2         9280
3        22415
4         6122
         ...  
48709    14586
48710    12543
48711    18948
48712    32413
48713    29295
Name: index_y, Length: 48714, dtype: int64

### Idea
Eliminate samples that are unique in terms of conditional data leaving only these responses that have few samples grouped by condtional data

Now we will have two conditional datasets and two response datasets

In [20]:
from sklearn.preprocessing import MinMaxScaler

In [21]:
data_cond

,Energy,Vx,Vy,Vz,Px,Py,Pz,mass,charge,std_proton,std_neutron,cond
0,3192.38,0.000000e+00,0.000000e+00,0.000000e+00,0.022422,-0.182957,-3192.38,939.565413,0.0,0.076311,0.632134,3192.38|0.0|0.0|0.0|0.022422400000000002|-0.18...
1,3961.55,0.000000e+00,0.000000e+00,0.000000e+00,-0.076487,0.179845,3961.55,938.272081,1.0,0.479201,0.127533,3961.55|0.0|0.0|0.0|-0.0764874|0.179845|3961.5...
2,2770.10,1.861170e-17,2.517190e-17,-1.689330e-13,0.305187,0.412760,-2770.10,497.611000,0.0,0.090233,0.779230,2770.1|1.86117e-17|2.5171900000000004e-17|-1.6...
3,3195.12,0.000000e+00,0.000000e+00,0.000000e+00,0.560528,-0.149980,3195.11,938.272081,1.0,0.131331,0.259529,3195.12|0.0|0.0|0.0|0.560528|-0.14998|3195.11|...
4,1714.07,0.000000e+00,0.000000e+00,0.000000e+00,0.457768,0.145639,-1714.07,939.565413,0.0,0.103628,0.355864,1714.07|0.0|0.0|0.0|0.457768|0.145639000000000...
...,...,...,...,...,...,...,...,...,...,...,...,...
48709,1456.66,7.796760e-18,2.269230e-18,-4.065040e-14,0.279388,0.081315,-1456.66,497.611000,0.0,0.062655,0.654967,1456.66|7.79676e-18|2.26923e-18|-4.06504e-14|0...
48710,3812.18,0.000000e+00,0.000000e+00,0.000000e+00,0.052953,0.402807,-3812.18,938.272081,1.0,0.362445,0.144962,3812.18|0.0|0.0|0.0|0.0529534|0.402807|-3812.1...
48711,1422.39,1.182560e-05,-1.385700e-06,3.655780e-02,0.511508,-0.052227,1422.39,0.000000,0.0,0.048635,0.105028,1422.39|1.1825599999999999e-05|-1.3857e-06|0.0...
48712,4071.23,0.000000e+00,0.000000e+00,0.000000e+00,0.089305,0.069410,-4071.23,1115.683000,0.0,0.422491,0.886010,4071.23|0.0|0.0|0.0|0.089305|0.0694104|-4071.2...


In [22]:
data_proton = np.log(data_proton+1)
data_proton = np.float32(data_proton)
print("data max", data_proton.max(), "min", data_proton.min())

data_proton_2 = data_proton[ids]

data_neutron = np.log(data_neutron+1)
data_neutron = np.float32(data_neutron)
print("data max", data_neutron.max(), "min", data_neutron.min())

data_neutron_2 = data_neutron[ids]

data_cond = data_cond.drop(columns="cond")

scaler_proton = MinMaxScaler()
std_proton = data_cond["std_proton"].values.reshape(-1,1)
std_proton = np.float32(std_proton)
std_proton = scaler_proton.fit_transform(std_proton)
print("std max", std_proton.max(), "min", std_proton.min())

scaler_neutron = MinMaxScaler()
std_neutron = data_cond["std_neutron"].values.reshape(-1,1)
std_neutron = np.float32(std_neutron)
std_neutron = scaler_neutron.fit_transform(std_neutron)
print("std max", std_neutron.max(), "min", std_neutron.min())

data max 6.5206213 min 0.0
data max 6.2785215 min 0.0
std max 1.0 min 0.0
std max 1.0 min 0.0


In [23]:
scaler = StandardScaler()
data_cond = np.float32(data_cond.drop(columns=["std_proton", "std_neutron"]))
data_cond = scaler.fit_transform(data_cond)
print("cond max", data_cond.max(), "min", data_cond.min())

cond max 21.432093 min -35.632454


In [24]:
x_train_p, x_test_p, x_train_p_2, x_test_p_2, x_train_n, x_test_n, x_train_n_2, x_test_n_2, y_train, y_test, std_proton_train, std_proton_test, std_neutron_train, std_neutron_test = train_test_split(
data_proton, data_proton_2, data_neutron, data_neutron_2, data_cond, std_proton, std_neutron, test_size=0.2, shuffle=False)
print(x_train_p.shape, x_test_p.shape, x_train_p_2.shape, x_test_p_2.shape, x_train_n.shape, x_test_n.shape, x_train_n_2.shape, x_test_n_2.shape)
print(y_train.shape, y_test.shape, std_proton_train.shape, std_proton_test.shape, std_neutron_train.shape, std_neutron_test.shape)

(38971, 56, 30) (9743, 56, 30) (38971, 56, 30) (9743, 56, 30) (38971, 44, 44) (9743, 44, 44) (38971, 44, 44) (9743, 44, 44)
(38971, 9) (9743, 9) (38971, 1) (9743, 1) (38971, 1) (9743, 1)


In [25]:
#save scales
if SAVE_EXPERIMENT_DATA:
    save_scales("Proton", scaler.mean_, scaler.scale_)

- x_train_p, x_test_p,
- x_train_p_2, x_test_p_2,
- x_train_n, x_test_n,
- x_train_n_2, x_test_n_2,
- y_train, y_test,
- std_proton_train, std_proton_test,
- std_neutron_train, std_neutron_test

In [26]:
BATCH_SIZE = 128

# Training dataset

# datasets that in each index contain two samples from the same conditional data
dataset_p = tf.data.Dataset.from_tensor_slices(x_train_p).batch(batch_size=BATCH_SIZE)
dataset_p_2 = tf.data.Dataset.from_tensor_slices(x_train_p_2).batch(batch_size=BATCH_SIZE)

dataset_n = tf.data.Dataset.from_tensor_slices(x_train_n).batch(batch_size=BATCH_SIZE)
dataset_n_2 = tf.data.Dataset.from_tensor_slices(x_train_n_2).batch(batch_size=BATCH_SIZE)

# conditional data
dataset_cond = tf.data.Dataset.from_tensor_slices(y_train).batch(batch_size=BATCH_SIZE)

# standard deviation for each conditional data samples
dataset_std_proton = tf.data.Dataset.from_tensor_slices(std_proton_train).batch(batch_size=BATCH_SIZE)
dataset_std_neutron = tf.data.Dataset.from_tensor_slices(std_neutron_train).batch(batch_size=BATCH_SIZE)

# shuffled conditional data
fake_cond = tf.data.Dataset.from_tensor_slices(y_train).shuffle(12800).batch(batch_size=BATCH_SIZE)

# zipped data
dataset_with_cond = tf.data.Dataset.zip((dataset_p, dataset_p_2,
                                         dataset_n, dataset_n_2,
                                         dataset_cond, dataset_std_proton, dataset_std_neutron, fake_cond)).shuffle(12800)

# Validation dataset

val_dataset_p = tf.data.Dataset.from_tensor_slices(x_test_p).batch(batch_size=BATCH_SIZE)
val_dataset_p_2 = tf.data.Dataset.from_tensor_slices(x_test_p_2).batch(batch_size=BATCH_SIZE)

val_dataset_n = tf.data.Dataset.from_tensor_slices(x_test_n).batch(batch_size=BATCH_SIZE)
val_dataset_n_2 = tf.data.Dataset.from_tensor_slices(x_test_n_2).batch(batch_size=BATCH_SIZE)

val_dataset_cond = tf.data.Dataset.from_tensor_slices(y_test).batch(batch_size=BATCH_SIZE)
val_dataset_std_proton = tf.data.Dataset.from_tensor_slices(std_proton_test).batch(batch_size=BATCH_SIZE)
val_dataset_std_neutron = tf.data.Dataset.from_tensor_slices(std_neutron_test).batch(batch_size=BATCH_SIZE)
val_fake_cond =  tf.data.Dataset.from_tensor_slices(y_test).shuffle(12800).batch(batch_size=BATCH_SIZE)

val_dataset_with_cond = tf.data.Dataset.zip((val_dataset_p, val_dataset_p_2,
                                             val_dataset_n, val_dataset_n_2,
                                             val_dataset_cond, val_dataset_std_proton, val_dataset_std_neutron, val_fake_cond)).shuffle(12800)

### Define Model

In [27]:
from tensorflow.compat.v1.keras.layers import Input, Dense, LeakyReLU, Conv2D, MaxPooling2D, UpSampling2D,  Concatenate
from tensorflow.compat.v1.keras.models import Model
from tensorflow.compat.v1.keras.layers import Dense, Reshape, Flatten
from tensorflow.compat.v1.keras.layers import Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import mse, binary_crossentropy, logcosh
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

from tensorflow import keras

In [28]:
# OLD Architecture

latent_dim = 8
cond_dim = 9

############################ generator ############################

x = Input(shape=(latent_dim,))
cond = Input(shape=(cond_dim,))
inputs = Concatenate(axis=1)([x, cond])


# PROTON HEAD
layer_1 = Dense(128*2)(inputs)
layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

layer_2 = Dense(128*20*12)(layer_1_a)
layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)

reshaped = Reshape((20,12,128))(layer_2_a)
reshaped_s = UpSampling2D(size=(3,2))(reshaped)

# PROTON HEAD
conv1 = Conv2D(256, kernel_size=(2, 6), padding='valid')(reshaped_s)
conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
conv1_a_s = UpSampling2D(size=(1,2))(conv1_a)

conv2 = Conv2D(128, kernel_size=(2, 6))(conv1_a_s)
conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)

conv3 = Conv2D(64, kernel_size=(2, 4))(conv2_a)
conv3_bd = Dropout(0.2)(BatchNormalization()(conv3))
conv3_a = LeakyReLU(alpha=0.1)(conv3_bd)

# NEUTRON HEAD
conv4 = Conv2D(256, kernel_size=(6, 2), padding='valid')(reshaped_s)
conv4_bd = Dropout(0.2)(BatchNormalization()(conv4))
conv4_a = LeakyReLU(alpha=0.1)(conv4_bd)
conv4_a_s = UpSampling2D(size=(1,2))(conv4_a)

conv5 = Conv2D(128, kernel_size=(6, 2))(conv4_a_s)
conv5_bd = Dropout(0.2)(BatchNormalization()(conv5))
conv5_a = LeakyReLU(alpha=0.1)(conv5_bd)

conv6 = Conv2D(64, kernel_size=(6, 2))(conv5_a)
conv6_bd = Dropout(0.2)(BatchNormalization()(conv6))
conv6_a = LeakyReLU(alpha=0.1)(conv6_bd)

outputs_proton = Conv2D(1, kernel_size=(2, 1), activation='relu')(conv3_a)
outputs_neutron = Conv2D(1, kernel_size=(2, 1), activation='relu')(conv6_a)

generator = Model([x, cond], [outputs_proton, outputs_neutron], name='generator')
generator.summary()

############################ discriminator ############################

# PROTON IMAGE
input_img_proton = Input(shape=[56, 30, 1], name='input_img_proton')
conv1 = Conv2D(32, kernel_size=3)(input_img_proton)
conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1_a)

conv2 = Conv2D(16, kernel_size=3)(pool1)
conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)
pool2 = MaxPooling2D(pool_size=(2, 1))(conv2_a)

flat_proton = Flatten()(pool2)

# NEUTRON IMAGE
input_img_neutron = Input(shape=[44, 44, 1], name='input_img_neutron')
conv3 = Conv2D(32, kernel_size=3)(input_img_neutron)
conv3_bd = Dropout(0.2)(BatchNormalization()(conv3))
conv3_a = LeakyReLU(alpha=0.1)(conv3_bd)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3_a)

conv4 = Conv2D(16, kernel_size=3)(pool3)
conv4_bd = Dropout(0.2)(BatchNormalization()(conv4))
conv4_a = LeakyReLU(alpha=0.1)(conv4_bd)
pool4 = MaxPooling2D(pool_size=(2, 1))(conv4_a)

flat_neutron = Flatten()(pool4)

# CONDITIONAL
cond = Input(shape=(cond_dim,))

inputs2 = Concatenate(axis=1)([flat_proton, flat_neutron, cond])

layer_1 = Dense(128)(inputs2)
layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

layer_2 = Dense(64)(layer_1_a)
layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)
outputs = Dense(2, activation='sigmoid')(layer_2_a)

discriminator = Model([input_img_proton, input_img_neutron, cond], [outputs, layer_2_a], name='discriminator')
discriminator.summary()

Model: "generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 9)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 17)           0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 256)          4608        ['concatenate[0][0]']    

### Idea
Increase number of latent dimensions with the old architecture

### IDEA
Why do we only take into account a loss of only single generated image

In [29]:
# def discriminator_loss(real_output, fake_output, fake_output_2):
#     real_loss = cross_entropy(tf.ones_like(real_output), real_output)
#     fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
#     fake_loss_2 = cross_entropy(tf.zeros_like(fake_output_2), fake_output_2)
#     total_loss = real_loss + fake_loss + fake_loss_2
    
#     # update state of accuracy of real and false images
#     d_acc_r.update_state(tf.ones_like(real_output), real_output)
#     d_acc_f.update_state(tf.zeros_like(fake_output), fake_output)
#     return total_loss

In [30]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    
    # update state of accuracy of real and false images
    d_acc_r.update_state(tf.ones_like(real_output), real_output)
    d_acc_f.update_state(tf.zeros_like(fake_output), fake_output)
    return total_loss

In [31]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [32]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)
d_acc_r = keras.metrics.BinaryAccuracy(name="d_acc_r", threshold=0.5)
d_acc_f = keras.metrics.BinaryAccuracy(name="d_acc_r", threshold=0.5)
g_acc = keras.metrics.BinaryAccuracy(name="g_acc_g", threshold=0.5)

### IDEA
Why do we only take into accoutn a single genrated image, not from two latent codes

In [33]:
# def generator_loss(step, fake_output, fake_output_2,
#                    fake_latent, fake_latent_2,
#                    noise, noise_2,
#                    std_proton, std_neutron):

#     g_acc.update_state(tf.ones_like(fake_output), fake_output)
#     g_acc.update_state(tf.ones_like(fake_output_2), fake_output_2)

#     crossentropy_loss = cross_entropy(tf.ones_like(fake_output), fake_output) + cross_entropy(tf.ones_like(fake_output_2), fake_output_2)

#     div = tf.math.divide(tf.reduce_mean(tf.abs(fake_latent - fake_latent_2),(1)), tf.reduce_mean(tf.abs(noise-noise_2),(1)))

#     div_loss_proton = std_proton * STRENGTH / (div + 1e-5)
#     div_loss_neutron = std_neutron * STRENGTH / (div + 1e-5)

#     div_loss_proton = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_proton,(1)), div_loss_proton))
#     div_loss_neutron = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_neutron,(1)), div_loss_neutron))
#     div_loss = div_loss_proton + div_loss_neutron
#     return crossentropy_loss + div_loss, div_loss

### IDEA
Should we take mean of diversity loss or just sum the up?

In [34]:
def generator_loss(step, fake_output,
                   fake_latent, fake_latent_2,
                   noise, noise_2,
                   std_proton, std_neutron):

    g_acc.update_state(tf.ones_like(fake_output), fake_output)

    crossentropy_loss = cross_entropy(tf.ones_like(fake_output), fake_output)

    div = tf.math.divide(tf.reduce_mean(tf.abs(fake_latent - fake_latent_2),(1)), tf.reduce_mean(tf.abs(noise-noise_2),(1)))

    div_loss_proton = std_proton * STRENGTH / (div + 1e-5)
    div_loss_neutron = std_neutron * STRENGTH / (div + 1e-5)

    div_loss_proton = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_proton,(1)), div_loss_proton))
    div_loss_neutron = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_neutron,(1)), div_loss_neutron))

    # average diversity loss
    div_loss = div_loss_proton + div_loss_neutron
    return crossentropy_loss + div_loss, div_loss

In [35]:
EPOCHS = 200
noise_dim = latent_dim
num_examples_to_generate = 16

START_GENERATING_IMG_FROM_IDX = 20
# Seed to reuse for generating samples for comparison during training
seed = tf.random.normal([num_examples_to_generate, noise_dim])
seed_cond = y_test[START_GENERATING_IMG_FROM_IDX:START_GENERATING_IMG_FROM_IDX+num_examples_to_generate]

In [36]:
wandb.finish()
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Generative Models for CERN Fast Simulations",
    name=wandb_run_name,
    # track hyperparameters and run metadata
    config={
    "Model": NAME,
    "dataset": "proton_neutron_data",
    "epochs": EPOCHS,
    "Date": DATE_STR,
    "latent_dimension": latent_dim,
    "Proton_min": photon_sum_proton_min,
    "Proton_max": photon_sum_proton_max,
    "Experiment_dir_name": EXPERIMENT_DIR_NAME,
    "batch_size": BATCH_SIZE
    },
    tags=[f"proton_min_{photon_sum_proton_min}",
          f"proton_max_{photon_sum_proton_max}",
          f"neutron_min_{photon_sum_neutron_min}",
          f"neutron_max_{photon_sum_neutron_max}",
          f"gan_strength_{STRENGTH}", "sdi-gan"]
)

wandb: Currently logged in as: bedkowski-patrick (nlp-wut-2023). Use `wandb login --relogin` to force relogin


In [37]:
from scipy.stats import wasserstein_distance
import pandas as pd
from utils import sum_channels_parallel
from sklearn.metrics import mean_absolute_error

org_p = np.exp(x_test_p)-1
ch_org_p = np.array(org_p).reshape(-1,56,30)
ch_org_p = pd.DataFrame(sum_channels_parallel(ch_org_p)).values
del org_p

org_n = np.exp(x_test_n)-1
ch_org_n = np.array(org_n).reshape(-1,44,44)
ch_org_n = pd.DataFrame(sum_channels_parallel(ch_org_n)).values
del org_n


def calculate_ws_ch(n_calc):
    ws_p = [0,0,0,0,0]
    ws_n = [0,0,0,0,0]
    for j in range(n_calc):
        z = np.random.normal(0,1,(x_test_p.shape[0], latent_dim))
        z_c = y_test
        results_p, results_n = generator.predict([z, z_c])

        results_p = np.exp(results_p)-1
        results_n = np.exp(results_n)-1
        try:
            ch_gen_p = np.array(results_p).reshape(-1,56,30)
            ch_gen_p = pd.DataFrame(sum_channels_parallel(ch_gen_p)).values
            ch_gen_n = np.array(results_n).reshape(-1,44,44)
            ch_gen_n = pd.DataFrame(sum_channels_parallel(ch_gen_n)).values
            for i in range(5):
                ws_p[i] = ws_p[i] + wasserstein_distance(ch_org_p[:,i], ch_gen_p[:,i])
                ws_n[i] = ws_n[i] + wasserstein_distance(ch_org_n[:,i], ch_gen_n[:,i])
            ws_p = np.array(ws_p)
            ws_n = np.array(ws_n)
            ws = ws_p+ws_n
        except ValueError as e:
            print(e)

    ws = ws/n_calc
    ws_mean = ws.sum()/5
    print("ws mean",f'{ws_mean:.2f}', end=" ")
    for n, score in enumerate(ws):
        print("ch"+str(n+1),f'{score:.2f}',end=" ")
    return ws_mean


In [38]:
@tf.function
def train_step(batch, step):
    # dataset proton, dataset 2 proton, dataset neutron, dataset 2 neutron, dataset_cond, dataset_std_proton, dataset_std_neutron, fake_cond
    images_p, images_p_2, images_n, images_n_2, cond, std_proton, std_neutron, noise_cond = batch
    step=step
    BATCH_SIZE = tf.shape(images_p)[0]
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    noise_2 = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # for the same conditional data generate two images from different noises
        generated_images_p, generated_images_n = generator([noise, noise_cond], training=True)
        generated_images_p_2, generated_images_n_2 = generator([noise_2, noise_cond], training=True)
        
        # produce if real image is real or fake
        real_output, real_latent = discriminator([images_p, images_n, cond], training=True)
        # real_output_2,real_latent_2  = discriminator([images_2,cond], training=True)
        
        # produce if generated images from two different latent codes are real or fake
        fake_output, fake_latent = discriminator([generated_images_p, generated_images_n, noise_cond], training=True)
        fake_output_2, fake_latent_2 = discriminator([generated_images_p_2, generated_images_n_2, noise_cond], training=True)

        gen_loss, div_loss = generator_loss(step, fake_output,
                                            fake_latent, fake_latent_2,
                                            noise, noise_2,
                                            std_proton, std_neutron)
        disc_loss = discriminator_loss(real_output, fake_output)

    #         generated_images = generator([noise,noise_cond], training=True)

    #         real_output = discriminator([images,cond], training=True)
    #         fake_output = discriminator([generated_images, noise_cond], training=True)

    #         gen_loss = generator_loss(step, fake_output)
    #         real_loss, fake_loss = discriminator_loss(real_output, fake_output)
    #         disc_loss = real_loss + fake_loss

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss, div_loss


In [39]:
# If model achieves WS metric less or equal to this number, its weights will be saved
WS_MEAN_SAVE_THRESHOLD = 20


if SAVE_EXPERIMENT_DATA:
    filepath_mod = f"../../{EXPERIMENT_DIR_NAME}/models/"
    create_dir(filepath_mod)

history = []
def train(dataset, epochs):
    experiment_start = time.time()
    tf_step = tf.Variable(0, dtype=float)
    step=0

    # generate first image
    generate_and_save_images(generator,
                             epochs,
                             [seed, seed_cond])

    for epoch in range(epochs):
        start = time.time()

        gen_loss_epoch = []
        div_loss_epoch = []
        disc_loss_epoch = []
        for batch in dataset:
            gen_loss, disc_loss, div_loss = train_step(batch,tf_step)

            history.append([gen_loss,disc_loss,
                100*d_acc_r.result().numpy(),
                100*d_acc_f.result().numpy(),
                100*g_acc.result().numpy(),
                ])
            tf_step.assign_add(1)
            step = step+1

            gen_loss_epoch.append(gen_loss)
            disc_loss_epoch.append(disc_loss)
            div_loss_epoch.append(div_loss)
            if step % 100 == 0:
                print("%d [D real acc: %.2f%%] [D fake acc: %.2f%%] [G acc: %.2f%%] "% (
                    step,
                    100*d_acc_r.result().numpy(),
                    100*d_acc_f.result().numpy(),
                    100*g_acc.result().numpy()))

        plot = generate_and_save_images(generator,
                                 epoch,
                                 [seed, seed_cond])

        ws_mean = calculate_ws_ch(min(epoch//5+1,5))

        if SAVE_EXPERIMENT_DATA:
            if ws_mean <= WS_MEAN_SAVE_THRESHOLD:
                # Save the model every epoch
                generator.compile()
                # discriminator.compile()
                generator.save((os.path.join(filepath_mod, "gen_"+NAME + "_"+ str(epoch) +".h5")))
                # discriminator.save((os.path.join(filepath_mod, "disc_"+NAME + "_"+ str(epoch) +".h5")))
                # np.savez(os.path.join(filepath_mod, "history_"+NAME+".npz"),np.array(history))

        wandb.log({
            'ws_mean': ws_mean,
            'gen_loss': np.mean(gen_loss_epoch),
            'div_loss': np.mean(div_loss_epoch),
            'disc_loss': np.mean(disc_loss_epoch),
            'epoch': epoch,
            'plot': wandb.Image(plot),
            'experiment_time': time.time()-experiment_start
        })

        plt.close('all')

        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    return history

In [40]:
if SAVE_EXPERIMENT_DATA:
    filepath_img = f"../../{EXPERIMENT_DIR_NAME}/images/"
    create_dir(filepath_img)

    
def generate_and_save_images(model, epoch, test_input):
    START_INDEX = 6
    SUPTITLE_TXT = f"\nModel: GAN proton data" \
               f"\nPhotonsum interval: [{photon_sum_proton_min}, {photon_sum_proton_max}]" \
               f"\nEPOCH: {epoch}"

    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions_p, predictions_n = model(test_input, training=False)
    """
    predictions has shape (n_samples, 56, 44, 2). First channel has proton data, second has neutrons
    """
    fig = plt.figure(figsize=(15,4))

    plt.title(f"EPOCH {epoch}")

    subfigs = fig.subfigures(1, 4)

    for particle_num, subfig in enumerate(subfigs.flat):  # iterate over 4 particles
        subfig.suptitle(f'Particle {particle_num} response')
        axs = subfig.subplots(2, 2)
        
        for i, ax in enumerate(axs.flat):  # iterate over 4 images of single particle
            m_2 = i % 2  # 0 if proton, 1 if neutron
            if i < 2:
                # Real response
                if m_2 == 0:  # proton
                    x = x_test_p[START_INDEX+particle_num].reshape(56, 30)
                else:  # neutron
                    x = x_test_n[START_INDEX+particle_num].reshape(44, 44)
                axs[i//2, m_2].set_title("neutron" if m_2 else "proton")
            else:
                # Generated response
                if m_2 == 0:  # proton
                    x = predictions_p[START_INDEX+particle_num].numpy().reshape(56, 30)
                else:  # neutron
                    x = predictions_n[START_INDEX+particle_num].numpy().reshape(44, 44)
            axs[i//2, m_2].set_axis_off()
            im = axs[i//2, m_2].imshow(x, interpolation='none', cmap='gnuplot')
            fig.colorbar(im, ax=axs[i//2, m_2])

    if SAVE_EXPERIMENT_DATA:
        plt.savefig(os.path.join(filepath_img, 'image_at_epoch_{:04d}.png'.format(epoch)))
    
    return fig

### Train model

In [ ]:
history = train(dataset_with_cond, EPOCHS)

100 [D real acc: 51.01%] [D fake acc: 56.15%] [G acc: 43.85%] 
200 [D real acc: 56.60%] [D fake acc: 55.91%] [G acc: 44.09%] 
300 [D real acc: 59.99%] [D fake acc: 55.51%] [G acc: 44.49%] 
305/305 [==============================] - 15s 46ms/step
ws mean 6757615662069491.00 ch1 28962905271.56 ch2 10874979.64 ch3 22500987538845308.00 ch4 159579117619304.03 ch5 11127482680102590.00 Time for epoch 1 is 449.2795214653015 sec
400 [D real acc: 60.86%] [D fake acc: 55.07%] [G acc: 44.93%] 
500 [D real acc: 61.04%] [D fake acc: 55.08%] [G acc: 44.92%] 
600 [D real acc: 61.36%] [D fake acc: 55.51%] [G acc: 44.49%] 
305/305 [==============================] - 13s 44ms/step
ws mean 3604.82 ch1 346.39 ch2 528.19 ch3 50.29 ch4 5083.40 ch5 12015.81 Time for epoch 2 is 410.713271856308 sec
700 [D real acc: 61.54%] [D fake acc: 55.92%] [G acc: 44.08%] 
800 [D real acc: 61.73%] [D fake acc: 55.94%] [G acc: 44.06%] 
900 [D real acc: 61.71%] [D fake acc: 56.12%] [G acc: 43.88%] 
305/305 [==================

In [ ]:
import seaborn as sns

sns.set_context('notebook', font_scale = 1)

In [ ]:
# total_loss, reconstruction_loss, kl_loss
history_losses = np.array([[float(loss) for loss in losses] for losses in history])

In [ ]:
generator_loss = history_losses[:,0]
disc_loss = history_losses[:,1]
discriminator_r_acc = history_losses[:,2]
discriminator_f_acc = history_losses[:,3]
generator_acc = history_losses[:,4]

In [ ]:
a4_dims = (14, 5)
def print_loss(loss_values, loss_str):
    fig, ax = plt.subplots(figsize=a4_dims)
    sns.lineplot(loss_values)
    plt.xscale('log')
    ax.set_title(f"{loss_str} loss in each epoch")
    ax.set_xlabel("Epoch number")
    ax.set_ylabel(f"{loss_str} loss value")
    # fig.save(f"../images/{EXPERIMENT_NAME}/{loss_str}_loss")

In [ ]:
print_loss(generator_loss, "Generator")
print_loss(disc_loss, "Discriminator")
print_loss(discriminator_r_acc, "Discriminator r accuracy")
print_loss(discriminator_f_acc, "Discriminator f accuracy")
print_loss(generator_acc, "Generator accuracy")